Import Libraries

In [5]:
from pyvis.network import Network
import networkx as nx
import random

Create Helper Functions

In [6]:
# recursive function that will traverse the nodes
def createPaper(network, authors, probStop):
    '''
    Will take network, list of authors, and probStop as input
    '''
    authorID = authors[-1]
    newNeighbors = set(network.neighbors(authorID)).difference(set(authors))

    # base condition: stop at node if probStop hit or there are no new neighbors to traverse
    if random.random() < probStop or len(newNeighbors) == 0:
        return
    
    # create list reprsenting probabilities for the neighboring nodes of the current coauthor
    probs = []
    for neighbor in newNeighbors:
        nData = network.get_edge_data(authorID, neighbor)
        probs.extend([neighbor] * nData["weight"])

    # Select coauthor from neighbors probabilities list
    try:
        coauthorID = random.choice(probs)
    except IndexError:
        print(f'Probs: {probs} and neighbors: {newNeighbors}')

    # update all edges of coauthors to this new author
    for author in authors:
        # if there is not an edge, create one
        if not network.has_edge(author, coauthorID) and author != coauthorID:
            network.add_edge(author, coauthorID, weight=0, width=1)
        newWeight = network.get_edge_data(authorID, coauthorID)["weight"] + 1
        #network.update(edges=[ (authorID, coauthorID, {"weight": newWeight, "width": newWeight//2}) ])
        network.update(edges=[ (authorID, coauthorID, {"weight": newWeight}) ])

    # call function recursively with coauthor
    authors.append(coauthorID)
    createPaper(network, authors, probStop)

def create():
    return

Define initial parameters

In [7]:
# define time steps
timeSteps = 50

# Probabilities
# probability that you generate new author
pw = 0.5

# define fields
fieldColors = {"CS": "blue", 
                "Math": "green", 
                "Physics": "red"}
fields = list(fieldColors.keys())

# define initial scholars, will be in form (id, scholarField, color)
scholarField = random.choice(fields)
nodeID = 0


Create Model

In [8]:
network = nx.Graph()
scholarField = random.choice(fields)
network.add_node(nodeID, label=scholarField, color=fieldColors[scholarField])
# go through time steps, add new scholar and paper at each step
for i in range(1, timeSteps):

    # Choose first author, either new scholar or random choice
    currNodes = list(network.nodes())
    authors = [random.choice(currNodes)]

    # with probability, add new author to network
    if random.random() < pw:
        # generate author and field
        scholarField = random.choice(fields)
        nodeID += 1
        author = nodeID
        network.add_node(author, label=scholarField, color=fieldColors[scholarField])

        # generate random coauthor from currNodes, which doesn't have the new node added in
        coauthorID = random.choice(currNodes)
        network.add_edge(author, coauthorID, weight=1, width=1)

        # update authors list
        authors = [author, coauthorID]

    # Add new paper, calling function
    probStop = 0.3
    createPaper(network, authors, probStop)

    # add authors as members to this topic

    # add emergence of new topic if field multidiscipiplinary (i.e. the union of the fields)

    # do you make it the author of the paper?

NameError: name 'props' is not defined

Display Network

In [ ]:
nt = Network()
# populates the nodes and edges data structures
nt.from_nx(network)
nt
nt.show('testModel.html')